(COMPPARALELOGPUSSMC)=

# 5.5 Cómputo en paralelo usando GPUs en un sistema de memoria compartida (SMC)

```{admonition} Notas para contenedor de docker:

Comando de docker para ejecución de la nota de forma local:

nota: cambiar `<ruta a mi directorio>` por la ruta de directorio que se desea mapear a `/datos` dentro del contenedor de docker.

`docker run --rm -v <ruta a mi directorio>:/datos --name jupyterlab_optimizacion_2 -p 8888:8888 -p 8787:8787 -d palmoreck/jupyterlab_optimizacion_2:3.0.0`

password para jupyterlab: `qwerty`

Detener el contenedor de docker:

`docker stop jupyterlab_optimizacion_2`

Documentación de la imagen de docker `palmoreck/jupyterlab_optimizacion_2:3.0.0` en [liga](https://github.com/palmoreck/dockerfiles/tree/master/jupyterlab/optimizacion_2).

```

---

Nota generada a partir de [liga](https://www.dropbox.com/s/yjijtfuky3s5dfz/2.5.Compute_Unified_Device_Architecture.pdf?dl=0)

```{admonition} Al final de esta nota el y la lectora:
:class: tip

*

```

Se presentan códigos y sus ejecuciones en una máquina `<falta colocar>` de la nube de [AWS](https://aws.amazon.com/). Se utilizó la AMI `opt2-aws-educate-openblas-04-04-2021` de la región `us-east-1` (Virginia) para reproducibilidad de resultados. Tal AMI se construyó a partir de una AMI `ubuntu 20.04 - ami-042e8287309f5df03` con el [script_profiling_and_BLAS.sh](https://github.com/palmoreck/scripts_for_useful_tools_installations/blob/main/AWS/ubuntu_20.04/optimizacion_2/script_profiling_and_BLAS.sh)

````{admonition} Comentario

Si se utiliza la *AMI* `opt2-aws-educate-openblas-04-04-2021` colocar en `User data` el siguiente *script*:

```bash

#!/bin/bash
##variables:
region=us-east-1 #make sure instance is in Virginia
name_instance=OpenBLAS
USER=ubuntu
##System update
apt-get update -yq
##Tag instance
INSTANCE_ID=$(curl -s http://instance-data/latest/meta-data/instance-id)
PUBLIC_IP=$(curl -s http://instance-data/latest/meta-data/public-ipv4)
sudo -H -u $USER bash -c "/home/$USER/.local/bin/aws ec2 create-tags --resources $INSTANCE_ID --tag Key=Name,Value=$name_instance-$PUBLIC_IP --region=$region"
sudo -H -u $USER bash -c "cd / && /home/$USER/.local/bin/jupyter lab --ip=0.0.0.0 --no-browser --config=/home/$USER/.jupyter/jupyter_notebook_config.py &"

```

````

La máquina `<falta colocar>` tiene las siguientes características:

**Falta ejecutar de acuerdo a la máquina elegida**

In [1]:
%%bash
lscpu

Architecture:                    x86_64
CPU op-mode(s):                  32-bit, 64-bit
Byte Order:                      Little Endian
Address sizes:                   46 bits physical, 48 bits virtual
CPU(s):                          64
On-line CPU(s) list:             0-63
Thread(s) per core:              2
Core(s) per socket:              16
Socket(s):                       2
NUMA node(s):                    2
Vendor ID:                       GenuineIntel
CPU family:                      6
Model:                           79
Model name:                      Intel(R) Xeon(R) CPU E5-2686 v4 @ 2.30GHz
Stepping:                        1
CPU MHz:                         2290.913
CPU max MHz:                     3000.0000
CPU min MHz:                     1200.0000
BogoMIPS:                        4600.03
Hypervisor vendor:               Xen
Virtualization type:             full
L1d cache:                       1 MiB
L1i cache:                       1 MiB
L2 cache:                        8

In [2]:
%%bash
sudo lshw -C memory

  *-firmware
       description: BIOS
       vendor: Xen
       physical id: 0
       version: 4.11.amazon
       date: 08/24/2006
       size: 96KiB
       capabilities: pci edd
  *-memory
       description: System Memory
       physical id: 1000
       size: 256GiB
       capabilities: ecc
       configuration: errordetection=multi-bit-ecc
     *-bank:0
          description: DIMM RAM
          physical id: 0
          slot: DIMM 0
          size: 16GiB
          width: 64 bits
     *-bank:1
          description: DIMM RAM
          physical id: 1
          slot: DIMM 1
          size: 16GiB
          width: 64 bits
     *-bank:2
          description: DIMM RAM
          physical id: 2
          slot: DIMM 2
          size: 16GiB
          width: 64 bits
     *-bank:3
          description: DIMM RAM
          physical id: 3
          slot: DIMM 3
          size: 16GiB
          width: 64 bits
     *-bank:4
          description: DIMM RAM
          physical id: 4
          slot: DIMM

In [3]:
%%bash
uname -ar #r for kernel, a for all

Linux ip-10-0-0-140 5.4.0-1038-aws #40-Ubuntu SMP Fri Feb 5 23:50:40 UTC 2021 x86_64 x86_64 x86_64 GNU/Linux


```{admonition} Observación
:class: tip

En la celda anterior se utilizó el comando de *magic* `%%bash`. Algunos comandos de *magic* los podemos utilizar también con `import`. Ver [ipython-magics](https://ipython.readthedocs.io/en/stable/interactive/magics.html#)

```

## *Compute Unified Device Architecture* (CUDA)

### Un poco de historia...

```{margin}

GPGPU es un término que se utilizó para referirse a la programación general en unidades de procesamiento gráfico. Hoy en día se conoce simplemente como *GPU programming*. Ver [General-purpose computing on graphics processing units](https://en.wikipedia.org/wiki/General-purpose_computing_on_graphics_processing_units).

```

La industria de videojuegos impulsó el desarrollo de las tarjetas gráficas a una velocidad sin precedente a partir del año 1999 para incrementar el nivel de detalle visual en los juegos de video. Alrededor del 2003 se planteó la posibilidad de utilizar las unidades de procesamiento gráfico para procesamiento en paralelo relacionado con aplicaciones distintas al ambiente de gráficas. A partir del 2006 la empresa [NVIDIA](https://www.nvidia.com/en-us/about-nvidia/) introdujo CUDA, una plataforma GPGPU y un modelo de programación que facilita el procesamiento en paralelo en las GPU's.

Desde el 2006, las tarjetas gráficas han creado una brecha significativa con las unidades de procesamiento, CPU's. Ver por ejemplo las gráficas que *NVIDIA* publica año tras año y que están relacionadas con el número de operaciones en punto flotante por segundo (FLOPS) y la transferencia de datos en la memoria RAM de la GPU: [from-graphics-processing-to-general-purpose-parallel-computing](https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#from-graphics-processing-to-general-purpose-parallel-computing).

```{margin}

La GPU y la CPU están conectadas por una interconexión de nombre [PCI](https://en.wikipedia.org/wiki/Conventional_PCI).

```

Hoy en día se continúa el desarrollo de GPU's con mayor RAM, con mayor capacidad de cómputo y mejor conectividad con la CPU. Estos avances han permitido resolver problemas con mejor exactitud que los resueltos con las CPU's, por ejemplo en el terreno de *deep learning* en reconocimiento de imágenes. Ver [ImageNet Classification with Deep Convolutional Neural Networks](https://papers.nips.cc/paper/4824-imagenet-classification-with-deep-convolutional-neural-networks.pdf), [2012: A Breakthrough Year for Deep Learning](https://medium.com/limitlessai/2012-a-breakthrough-year-for-deep-learning-2a31a6796e73).



```{admonition} Observación
:class: tip

Para avances al día de hoy ver [NVIDIA Turing Architecture In-Depth](https://devblogs.nvidia.com/nvidia-turing-architecture-in-depth/), [samsung-amd-rdna-gpu-2021](https://wccftech.com/samsung-amd-rdna-gpu-2021/), [playstation-5-specifications-revealed-but-design-is-still-a-mystery](https://www.theguardian.com/games/2020/mar/19/playstation-5-specifications-revealed-but-design-is-still-a-mystery), [xbox-series-x-tech](https://news.xbox.com/en-us/2020/03/16/xbox-series-x-tech/) y recientemente [IBM Supercomputer Summit Attacks Coronavirus…](https://www.ibm.com/blogs/nordic-msp/ibm-supercomputer-summit-attacks-coronavirus/). 

```

```{margin}

Recuérdese la [taxonomía de Flynn](https://en.wikipedia.org/wiki/Flynn%27s_taxonomy).

```

La arquitectura en la que podemos ubicar a las GPU's es en la de un sistema MIMD y SIMD. De hecho es [SIMT: Simple Instruction Multiple Thread](https://en.wikipedia.org/wiki/Single_instruction,_multiple_threads) en un modelo de sistema de memoria compartida pues "los *threads* en un *warp* leen la misma instrucción para ser ejecutada".

```{admonition} Definición

Un *warp* en el contexto de GPU *programming* es un conjunto de *threads*. Equivale a $32$ *threads*.

```

### ¿Diferencia con la CPU multicore?


<img src="https://dl.dropboxusercontent.com/s/k11qub01w4nvksi/CPU_multicore.png?dl=0" heigth="500" width="500">

**GPU**

<img src="https://dl.dropboxusercontent.com/s/lw9kia12qhwp95r/GPU.png?dl=0" heigth="500" width="500">

```{admonition} Observación
:class: tip

Obsérvese en el dibujo anterior la diferencia en tamaño del caché en la CPU y GPU. También la unidad de control es más pequeña en la GPU.


```

```{margin}

Una máquina *quad core* soporta cuatro threads en cada *core*.

```

A diferencia de una máquina *multicore* o multi CPU's con la habilidad de lanzar en un instante de tiempo unos cuantos *threads*, la GPU puede lanzar cientos o miles de threads en un instante siendo cada core *heavily multithreaded*. Sí hay restricciones en el número de threads que se pueden lanzar en un instante pues las tarjetas gráficas tienen diferentes características (modelo) y arquitecturas, pero la diferencia es grande. Por ejemplo, la serie **GT 200** (2009) en un instante puede lanzar 30,720 threads en sus 240 *cores*. Ver [GeForce_200_series](https://en.wikipedia.org/wiki/GeForce_200_series), [List of NVIDIA GPU's](https://en.wikipedia.org/wiki/List_of_Nvidia_graphics_processing_units).

Ver [How Graphics Cards Work](https://computer.howstuffworks.com/graphics-card1.htm) y [How Microprocessors Work](https://computer.howstuffworks.com/microprocessor.htm) para más información.

### ¿Otras compañías producen tarjetas gráficas?

Ver por ejemplo la lista de GPU's de [Advanced Micro Devices](https://en.wikipedia.org/wiki/List_of_AMD_graphics_processing_units)

### ¿Si tengo una tarjeta gráfica de AMD puedo correr un programa de CUDA?

No es posible pero entre las alternativas están:

* [OpenCl](https://www.khronos.org/opencl/)

* [OpenACC](https://www.openacc.org/about)

### ¿Si tengo una tarjeta gráfica de NVIDIA un poco antigua puedo correr un programa de CUDA?

Las GPU's producidas por NVIDIA desde 2006 son capaces de correr programas basados en ***CUDA C***. La cuestión sería revisar qué *compute capability* tiene tu tarjeta. Ver [Compute Capabilities](https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#compute-capabilities) para las características que tienen las tarjetas más actuales.

### ¿Qué es *CUDA C*?

Es una extensión al lenguaje *C* de programación en el que se utiliza una nueva sintaxis para procesamiento en la GPU. Contiene también una librería *runtime* que define funciones que se ejecutan desde el ***host*** por ejemplo para alojar y desalojar memoria en el ***device***, transferir datos entre la memoria *host* y la memoria *device* o manejar múltiples devices. La librería *runtime* está hecha encima de una API de *C* de bajo nivel llamada [NVIDIA CUDA Driver API](https://docs.nvidia.com/cuda/cuda-driver-api/index.html) la cual es accesible desde el código. Para información de la API de la librería runtime ver [NVIDIA CUDA Runtime API](https://docs.nvidia.com/cuda/cuda-runtime-api/index.html).

```{admonition} Comentario

La transferencia de datos entre la memoria del *host* a *device* o viceversa es un *bottleneck* fuerte.

```

### ¿A qué se refiere la terminología de *host* y *device*?

*Host* es la máquina *multicore* CPU y *device* es la GPU. Una máquina puede tener múltiples GPU's por lo que tendrá múltiples *devices*.

### Tengo una tarjeta NVIDIA CUDA *capable* ¿qué debo realizar primero?

Realizar instalaciones dependiendo de tu sistema operativo. Ver [Instalación](https://github.com/ITAM-DS/analisis-numerico-computo-cientifico/tree/master/C/extensiones_a_C/CUDA/instalacion) donde además se encontrará información para instalación de [nvidia-docker](https://github.com/NVIDIA/nvidia-docker).

### Instalé lo necesario y al ejecutar en la terminal `nvcc -V` obtengo la versión... ¿cómo puedo probar mi instalación?

1) Obteniendo información del NVIDIA driver ejecutando en la terminal.

In [ ]:
%%bash
nvidia-smi

2) Compilando y ejecutando el siguiente programa de *CUDA C*:

%%file hello_world.cu

#include<stdio.h>
__global__ void func(void){
    printf("Hello world del bloque %d del thread %d!\n", blockIdx.x, threadIdx.x);
}
int main(void){
    func<<<2,3>>>(); //2 bloques de 3 threads cada uno
    cudaDeviceSynchronize();
    printf("Hola del cpu thread\n");
    return 0;
}


In [ ]:
%%bash
nvcc hello_world.cu -o hello_world.out

In [ ]:
%%bash
./hello_world.out

3) Haciendo un query a la GPU para ver qué características tiene (lo siguiente es posible ejecutar sólo si se instaló el *CUDA toolkit*): 

In [ ]:
%%bash
cd /usr/local/cuda/samples/1_Utilities/deviceQuery/ && sudo make
/usr/local/cuda/samples/1_Utilities/deviceQuery/deviceQuery

### ¿Por qué usar CUDA y *CUDA-C* o más general cómputo en la GPU?

* NVIDIA como se mencionó al inicio de la nota fue de las primeras compañías en utilizar la GPU para tareas no relacionadas con el área de gráficos y ha colaborado en el avance del conocimiento de las GPU's y desarrollo de algoritmos y tarjetas gráficas. Otra compañía es [Khronos_Group](https://en.wikipedia.org/wiki/Khronos_Group) por ejemplo, quien actualmente desarrolla [OpenCl](https://www.khronos.org/opencl/).

```{margin}

*Deep learning* se ha utilizado para resolver problemas en *machine learning* típicos. Ejemplos de esto son la clasificación de imágenes, de sonidos o análisis de textos, ver por ejemplo [Practical text analysis using deep learning](https://medium.com/@michael.fire/practical-text-analysis-using-deep-learning-5fb0744efdf9).

```

* El cómputo en la GPU constituye hoy en día (2020) una alternativa fuerte a la implementación de modelos de machine learning ampliamente utilizada por la comunidad científica. En el terreno de cómputo matricial y *deep learning* se encuentran [tensorflow](https://github.com/tensorflow), [caffe](https://github.com/BVLC/caffe), [Theano](https://github.com/Theano/Theano) (aunque ya no es soportado pero está retomado en [pymc-devs/aesara](https://github.com/pymc-devs/aesara)), [Pytorch](https://github.com/pytorch/pytorch) y [keras](https://github.com/keras-team/keras) como ejemplos de lo anterior. 

* Sí hay publicaciones científicas para la implementación de *deep learning* en las CPU's, ver por ejemplo el paper reciente de [SLIDE](https://www.cs.rice.edu/~as143/Papers/SLIDE_MLSys.pdf), [HashingDeepLearning](https://github.com/keroro824/HashingDeepLearning) y las entradas [An algorithm could make CPUs a cheap way to train AI](https://www.engadget.com/2020/03/03/rice-university-slide-cpu-gpu-machine-learning/) y [Deep learning rethink overcomes major obstacle in AI industry](https://www.sciencedaily.com/releases/2020/03/200305135041.htm) . Sin embargo, esta área se encuentra en activa investigación por lo que se han adoptado el uso de implementaciones del *deep learning* utilizando GPU's. 


```{admonition} Comentario

El paper plantea una discusión a realizar con la frase *...change in the state-of-the-art algorithms can render specialized hardware less effective in the future*. Ver por ejemplo [Tensor Cores](https://developer.nvidia.com/tensor-cores), [NVIDIA TENSOR CORES, The Next Generation of Deep Learning](https://www.nvidia.com/en-us/data-center/tensorcore/), [The most powerful computers on the planet: SUMMIT](https://www.ibm.com/thought-leadership/summit-supercomputer/) como ejemplos de hardware especializado en aprendizaje automático con Tensorflow.

*Summit powered by 9,126 IBM Power9 CPUs and over 27,000 NVIDIA V100 Tensor Core GPUS, is able to do 200 quadrillion calculations per second...* [IBM Supercomputer Summit Attacks Coronavirus…](https://www.ibm.com/blogs/nordic-msp/ibm-supercomputer-summit-attacks-coronavirus/).

```

## [CUDA-C](https://docs.nvidia.com/cuda/cuda-c-programming-guide/)

Consiste en extensiones al lenguaje C y en una *runtime library*.

### Kernel

* En CUDA C se define una función que se ejecuta en el ***device*** y que se le nombra ***kernel***. El *kernel* inicia con la sintaxis:

```C
__global__ void mifun(int param){
...
}

```

Siempre es tipo `void` (no hay `return`).

* El llamado al *kernel* se realiza desde el **host** y con una sintaxis en la que se define el número de threads, llamados **CUDA threads** (que son distintos a los *CPU threads*), y bloques, nombrados **CUDA blocks**, que serán utilizados para la ejecución del kernel. La sintaxis que se utiliza es con `<<< >>>` y en la primera entrada se coloca el número de *CUDA blocks* y en la segunda entrada el número de *CUDA threads*:


```C
__global__ void mifun(int param){
...
}

int main(){
    int par=0;
    mifun<<<N,5>>> (par); //N bloques de 5 threads
}

```

### Ejemplo

`hello_world.cu`

In [1]:
%%file hello_world.cu
#include<stdio.h>
__global__ void func(void){
}
int main(void){
    func<<<1,1>>>(); //1 bloque de 1 thread
    printf("Hello world!\n");
return 0;
}

Writing hello_world.cu


Compilación:

In [2]:
%%bash
nvcc --compiler-options -Wall hello_world.cu -o hello_world.out

Ejecución:

In [3]:
%%bash
./hello_world.out

Hello world!


```{admonition} Comentarios

* La función `main` se ejecuta en la CPU.

* `func` es un *kernel* y es ejecutada por los *CUDA threads* en el *device*. Obsérvese que tal función inicia con la sintaxis `__global__`. En este caso el *CUDA thread* que fue lanzado no realiza ninguna acción pues el cuerpo del kernel está vacío.

* El *kernel* sólo puede tener un `return` tipo *void*: `__global__ void func` por lo que el *kernel* debe regresar sus resultados a través de sus argumentos.
 
* `nvcc` es un *wrapper* para el compilador de programas escritos en `C`. El compilador instalado en el contenedor de docker descrito al inicio de ésta nota es `gcc`. 
 
* La extensión del archivo debe ser `.cu` aunque esto puede modificarse al compilar con `nvcc`: 

`$nvcc -x cu hello_world.c -o hello_world.out`

* En ocasiones para tener funcionalidad de un determinado [compute capability](https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#compute-capabilities) se especifica la *flag* de `-arch=sm_11` en la línea de `nvcc`. En este caso se le indica al compilador que compile el programa para un *compute capability* de $1.1$. Ver [run a kernel using the larger grid size support offered](https://stackoverflow.com/questions/16954931/cuda-5-0-cudagetdeviceproperties-strange-grid-size-or-a-bug-in-my-code) y [cuda-how-to-use-arch-and-code-and-sm-vs-compute](https://stackoverflow.com/questions/35656294/cuda-how-to-use-arch-and-code-and-sm-vs-compute) para más sobre esto.

```

### ¿Bloques de threads? 

Los *CUDA threads* son divididos en *CUDA blocks* y éstos se encuentran en un *grid*. En el lanzamiento del *kernel* se debe especificar al hardware cuántos *CUDA blocks* tendrá nuestro *grid* y cuántos *CUDA threads* estarán en cada bloque. 

### Ejemplo

In [4]:
%%file hello_world_2.cu
#include<stdio.h>
__global__ void func(void){
    printf("Hello world del bloque %d del thread %d!\n", blockIdx.x, threadIdx.x);
}
int main(void){
    func<<<2,3>>>(); //2 bloques de 3 threads cada uno
    cudaDeviceSynchronize();
    printf("Hola del cpu thread\n");
    return 0;
}


Writing hello_world_2.cu


In [5]:
%%bash 
nvcc --compiler-options -Wall hello_world_2.cu -o hello_world_2.out

In [6]:
%%bash
./hello_world_2.out

Hello world del bloque 0 del thread 0!
Hello world del bloque 0 del thread 1!
Hello world del bloque 0 del thread 2!
Hello world del bloque 1 del thread 0!
Hello world del bloque 1 del thread 1!
Hello world del bloque 1 del thread 2!
Hola del cpu thread


```{admonition} Comentarios

* En lo que continúa de la nota el nombre *thread* hará referencia a *CUDA thread* y el nombre bloque a *CUDA block*.

* El llamado a la ejecución del kernel se realizó en el *host* y se lanzaron $2$ bloques (primera posición en la sintaxis <<<>>>), cada uno con $3$ *threads*.

* Se utiliza la función [cudaDeviceSynchronize](https://docs.nvidia.com/cuda/cuda-runtime-api/group__CUDART__DEVICE.html#group__CUDART__DEVICE_1g10e20b05a95f638a4071a655503df25d) para que el *cpu-thread* espere la finalización de la ejecución del kernel.

* En el ejemplo anterior, las variables `blockIdx` y `threadIdx` hacen referencia a los **id**'s que tienen los bloques y los threads: el *id* del bloque dentro del *grid* y el *id* del thread dentro del bloque. La parte `.x` de las variables: `blockIdx.x` y `threadIdx.x` refieren a la **primera coordenada** del bloque en el *grid* y a la **primera coordenada** del *thread* en en el bloque. 

* La elección del número de bloques en un grid o el número de *threads* en un bloque no corresponde a alguna disposición del hardware, esto es, si se lanza un kernel con `<<< 1, 3 >>>` no implica que la GPU tenga en su hardware un bloque o 3 *threads*. Asimismo, las coordenadas que se obtienen vía `blockIdx` o `threadIdx` son meras abstracciones, no corresponden a algún ordenamiento en el hardware de la GPU.

* Todos los *threads* de un bloque  ejecutan el kernel por lo que se tienen tantas copias del kernel como número de bloques sean lanzados. Esto es una muestra la GPU sigue el modelo  *Single Instruction Multiple Threads [(SIMT)](https://en.wikipedia.org/wiki/Single_instruction,_multiple_threads)*.

```

### ¿Grid's y bloques 3-dimensionales?

En el *device* podemos definir el *grid* de bloques y el bloque de *threads* utilizando el tipo de dato `dim3` el cual también es parte de *CUDA C*:

### Ejemplo

In [7]:
%%file hello_world_3.cu
#include<stdio.h>
__global__ void func(void){
    printf("Hello world del bloque %d del thread %d!\n", blockIdx.y, threadIdx.z);
}
int main(void){
    dim3 dimGrid(1,2,1); //2 bloques en el grid
    dim3 dimBlock(1,1,3); //3 threads por bloque
    func<<<dimGrid,dimBlock>>>(); 
    cudaDeviceSynchronize();
    printf("Hola del cpu thread\n");
    return 0;
}

Writing hello_world_3.cu


In [8]:
%%bash 
nvcc --compiler-options -Wall hello_world_3.cu -o hello_world_3.out

In [9]:
%%bash
./hello_world_3.out

Hello world del bloque 0 del thread 0!
Hello world del bloque 0 del thread 1!
Hello world del bloque 0 del thread 2!
Hello world del bloque 1 del thread 0!
Hello world del bloque 1 del thread 1!
Hello world del bloque 1 del thread 2!
Hola del cpu thread


### Ejemplo

In [10]:
%%file thread_idxs.cu
#include<stdio.h>
__global__ void func(void){
    if(threadIdx.x==0 && threadIdx.y==0 && threadIdx.z==0){
        printf("blockIdx.x:%d\n",blockIdx.x);
    }
    printf("thread idx.x:%d\n",threadIdx.x);
    printf("thread idx.y:%d\n",threadIdx.y);
    printf("thread idx.z:%d\n",threadIdx.z);
}
int main(void){
    dim3 dimGrid(1,1,1); //1 bloque en el grid
    dim3 dimBlock(1,3,1); //3 threads por bloque
    func<<<dimGrid,dimBlock>>>(); 
    cudaDeviceSynchronize();
    return 0;
}

Writing thread_idxs.cu


In [11]:
%%bash 
nvcc --compiler-options -Wall thread_idxs.cu -o thread_idxs.out

In [12]:
%%bash
./thread_idxs.out

blockIdx.x:0
thread idx.x:0
thread idx.x:0
thread idx.x:0
thread idx.y:0
thread idx.y:1
thread idx.y:2
thread idx.z:0
thread idx.z:0
thread idx.z:0


### Ejemplo

In [13]:
%%file block_idxs.cu
#include<stdio.h>
__global__ void func(void){
    printf("blockIdx.x:%d\n",blockIdx.x);
    printf("blockIdx.y:%d\n",blockIdx.y);
    printf("blockIdx.z:%d\n",blockIdx.z);

}
int main(void){
    dim3 dimGrid(1,2,2); //4 bloques en el grid
    dim3 dimBlock(1,1,1); //1 thread por bloque
    func<<<dimGrid,dimBlock>>>(); 
    cudaDeviceSynchronize();
    return 0;
}

Writing block_idxs.cu


In [14]:
%%bash 
nvcc --compiler-options -Wall block_idxs.cu -o block_idxs.out

In [15]:
%%bash
./block_idxs.out

blockIdx.x:0
blockIdx.x:0
blockIdx.x:0
blockIdx.x:0
blockIdx.y:1
blockIdx.y:0
blockIdx.y:0
blockIdx.y:1
blockIdx.z:0
blockIdx.z:0
blockIdx.z:1
blockIdx.z:1


### Ejemplo

Podemos usar la variable `blockDim` para cada coordenada `x, y` o `z` y obtener la dimensión de los bloques:

In [16]:
%%file block_dims.cu
#include<stdio.h>
__global__ void func(void){
    if(threadIdx.x==0 && threadIdx.y==0 && threadIdx.z==0 && blockIdx.z==1){
    printf("blockDim.x:%d\n",blockDim.x);
    printf("blockDim.y:%d\n",blockDim.y);
    printf("blockDim.z:%d\n",blockDim.z);
    }

}
int main(void){
    dim3 dimGrid(2,2,2); //8 bloques en el grid
    dim3 dimBlock(3,1,2); //6 threads por bloque
    func<<<dimGrid,dimBlock>>>(); 
    cudaDeviceSynchronize();
    return 0;
}

Writing block_dims.cu


In [17]:
%%bash 
nvcc --compiler-options -Wall block_dims.cu -o block_dims.out

In [18]:
%%bash
./block_dims.out

blockDim.x:3
blockDim.x:3
blockDim.x:3
blockDim.x:3
blockDim.y:1
blockDim.y:1
blockDim.y:1
blockDim.y:1
blockDim.z:2
blockDim.z:2
blockDim.z:2
blockDim.z:2


### Alojamiento de memoria en el *device*

Para alojar memoria en el *device* se utiliza el llamado a [cudaMalloc](https://docs.nvidia.com/cuda/cuda-runtime-api/group__CUDART__MEMORY.html#group__CUDART__MEMORY_1g37d37965bfb4803b6d4e59ff26856356) y para transferir datos del *host* al *device* o viceversa se llama a lafunción [cudaMemcpy](https://docs.nvidia.com/cuda/cuda-runtime-api/group__CUDART__MEMORY.html#group__CUDART__MEMORY_1gc263dbe6574220cc776b45438fc351e8) con respectivos parámetros como `cudaMemcpyHostToDevice` o `cudaMemcpyDeviceToHost`. 

Para desalojar memoria del *device* se utiliza el llamado a [cudaFree](https://docs.nvidia.com/cuda/cuda-runtime-api/group__CUDART__MEMORY.html#group__CUDART__MEMORY_1ga042655cbbf3408f01061652a075e094).

### Ejemplo

**N bloques de 1 thread**

In [19]:
%%file suma_vectorial.cu
#include<stdio.h>
#define N 10
__global__ void suma_vect(int *a, int *b, int *c){
    int block_id_x = blockIdx.x;
    if(block_id_x<N) //aquí se asume que el valor de N 
                     //es menor al número máximo de bloques que se pueden lanzar
                     //si fuese mayor, hay que hacer un ajuste
        c[block_id_x] = a[block_id_x]+b[block_id_x];
}
int main(void){
    int a[N], b[N],c[N];
    int *device_a, *device_b, *device_c;
    int i;
    dim3 dimGrid(N,1,1); //N bloques en el grid
    dim3 dimBlock(1,1,1); //1 threads por bloque 
    //alojando en device
    cudaMalloc((void **)&device_a, sizeof(int)*N); 
    cudaMalloc((void **)&device_b, sizeof(int)*N);
    cudaMalloc((void **)&device_c, sizeof(int)*N);
    //llenando los arreglos con datos dummy:
    for(i=0;i<N;i++){
        a[i]=i;
        b[i]=i*i;
    }
    //copiamos arreglos a, b a la GPU
    cudaMemcpy(device_a,a,N*sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(device_b,b,N*sizeof(int), cudaMemcpyHostToDevice);
    //mandamos a llamar a suma_vect:
    suma_vect<<<dimGrid,dimBlock>>>(device_a,device_b,device_c); //N bloques de 1 thread
    cudaDeviceSynchronize();
    //copia del resultado al arreglo c:
    cudaMemcpy(c,device_c,N*sizeof(int),cudaMemcpyDeviceToHost);
    for(i=0;i<N;i++)
        printf("%d+%d = %d\n",a[i],b[i],c[i]);
    cudaFree(device_a);
    cudaFree(device_b);
    cudaFree(device_c);
    return 0;
}

Writing suma_vectorial.cu


In [20]:
%%bash
nvcc --compiler-options -Wall suma_vectorial.cu -o suma_vectorial.out

In [21]:
%%bash
./suma_vectorial.out

0+0 = 0
1+1 = 2
2+4 = 6
3+9 = 12
4+16 = 20
5+25 = 30
6+36 = 42
7+49 = 56
8+64 = 72
9+81 = 90


```{admonition} Comentarios

* Los *statements*:

```C
    int *device_a, *device_b, *device_c;
```

en sintaxis de *C* se definen apuntadores que refieren a una dirección de memoria. En el contexto de la *GPU programming* estos apuntadores no apuntan a una dirección de memoria en el *device*. Aunque NVIDIA añadió el *feature* de [Unified Memory](https://devblogs.nvidia.com/unified-memory-cuda-beginners/) (un espacio de memoria accesible para el *host* y el *device*) aquí no se está usando tal *feature*. Más bien se están utilizando los apuntadores anteriores para apuntar a un [struct](https://en.wikipedia.org/wiki/Struct_(C_programming_language)) de *C* en el que uno de sus tipos de datos es una dirección de memoria en el *device*.

* El uso de `(void **)` es por la definición de la función `cudaMalloc`.

* En el programa anterior se coloca en comentario que se asume que $N$ el número de datos en el arreglo es menor al número de bloques que es posible lanzar. Esto como veremos más adelante es importante considerar pues aunque en un *device* se pueden lanzar muchos bloques y muchos threads, se tienen límites en el número de éstos que es posible lanzar.


```

### ¿Perfilamiento en CUDA?

Al instalar el *CUDA toolkit* en sus máquinas o bien si utilizan el contenedor de docker (descrito al inicio de la nota) se instala la línea de comando [nvprof](https://docs.nvidia.com/cuda/profiler-users-guide/index.html) para perfilamiento. Se puede ejecutar con:

In [22]:
%%bash 
nvprof --normalized-time-unit s ./suma_vectorial.out

0+0 = 0
1+1 = 2
2+4 = 6
3+9 = 12
4+16 = 20
5+25 = 30
6+36 = 42
7+49 = 56
8+64 = 72
9+81 = 90


==342== NVPROF is profiling process 342, command: ./suma_vectorial.out
==342== Profiling application: ./suma_vectorial.out
==342== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
                        %         s                   s         s         s
 GPU activities:    36.80  1.47e-06         1  1.47e-06  1.47e-06  1.47e-06  suma_vect(int*, int*, int*)
                    35.20  1.41e-06         2  7.04e-07  5.44e-07  8.64e-07  [CUDA memcpy HtoD]
                    28.00  1.12e-06         1  1.12e-06  1.12e-06  1.12e-06  [CUDA memcpy DtoH]
      API calls:    99.55  0.087959         3  0.029320  4.69e-06  0.087948  cudaMalloc
                     0.25  2.18e-04        97  2.25e-06  1.79e-07  1.36e-04  cuDeviceGetAttribute
                     0.06  5.50e-05         3  1.83e-05  3.40e-06  4.55e-05  cudaFree
                     0.05  4.70e-05         1  4.70e-05  4.70e-05  4.70e-05  cuDeviceTotalMem
                     0.03  2.6

**Comentarios:**

* Las unidades en las que se reporta son s: second, ms: millisecond, us: microsecond, ns: nanosecond.

* En la documentación de NVIDIA se menciona que `nvprof` será reemplazada próximamente por [NVIDIA Nsight Compute](https://developer.nvidia.com/nsight-compute) y [NVIDIA Nsight Systems](https://developer.nvidia.com/nsight-systems).


En el ejemplo anterior se lanzaron $N$ bloques con $1$ *thread* cada uno y a continuación se lanza $1$ bloque con $N$ *threads*.

In [23]:
%%file suma_vectorial_2.cu
#include<stdio.h>
#define N 10
__global__ void suma_vect(int *a, int *b, int *c){
    int thread_id_x = threadIdx.x;
    if(thread_id_x<N) //aquí se asume que el valor de N 
                     //es menor al número máximo de threads que se pueden lanzar
                    //si fuese mayor, hay que hacer un ajuste
        c[thread_id_x] = a[thread_id_x]+b[thread_id_x];
}
int main(void){
    int *device_a, *device_b, *device_c;
    int i;
    dim3 dimGrid(1,1,1); //1 bloques en el grid
    dim3 dimBlock(N,1,1); //N threads por bloque 
    //alojando en device con Unified Memory
    cudaMallocManaged(&device_a, sizeof(int)*N);
    cudaMallocManaged(&device_b, sizeof(int)*N);
    cudaMallocManaged(&device_c, sizeof(int)*N);
    //llenando los arreglos:
    for(i=0;i<N;i++){
        device_a[i]=i;
        device_b[i]=i*i;
    }
    suma_vect<<<dimGrid,dimBlock>>>(device_a,device_b,device_c); //1 bloque con N threads
    cudaDeviceSynchronize();
    for(i=0;i<N;i++)
        printf("%d+%d = %d\n",device_a[i],device_b[i],device_c[i]);
    cudaFree(device_a);
    cudaFree(device_b);
    cudaFree(device_c);
    return 0;
}

Writing suma_vectorial_2.cu


In [24]:
%%bash
nvcc --compiler-options -Wall suma_vectorial_2.cu -o suma_vectorial_2.out

In [25]:
%%bash 
nvprof --normalized-time-unit s ./suma_vectorial_2.out

0+0 = 0
1+1 = 2
2+4 = 6
3+9 = 12
4+16 = 20
5+25 = 30
6+36 = 42
7+49 = 56
8+64 = 72
9+81 = 90


==389== NVPROF is profiling process 389, command: ./suma_vectorial_2.out
==389== Profiling application: ./suma_vectorial_2.out
==389== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
                        %         s                   s         s         s
 GPU activities:   100.00  2.05e-06         1  2.05e-06  2.05e-06  2.05e-06  suma_vect(int*, int*, int*)
      API calls:    99.40  0.151464         3  0.050488  1.09e-05  0.151432  cudaMallocManaged
                     0.27  4.08e-04         1  4.08e-04  4.08e-04  4.08e-04  cudaLaunchKernel
                     0.19  2.82e-04        97  2.91e-06  3.09e-07  1.61e-04  cuDeviceGetAttribute
                     0.06  9.38e-05         3  3.13e-05  1.15e-05  5.90e-05  cudaFree
                     0.05  7.66e-05         1  7.66e-05  7.66e-05  7.66e-05  cuDeviceTotalMem
                     0.02  2.95e-05         1  2.95e-05  2.95e-05  2.95e-05  cuDeviceGetName
                     0.0

**Comentarios:**

* El programa anterior utiliza la [Unified Memory](https://devblogs.nvidia.com/unified-memory-cuda-beginners/) con la función [cudaMallocManaged](https://docs.nvidia.com/cuda/cuda-runtime-api/group__CUDART__HIGHLEVEL.html#group__CUDART__HIGHLEVEL_1gcf6b9b1019e73c5bc2b39b39fe90816e). La *Unified Memory* es un *feature* que se añadió a CUDA desde las arquitecturas de **Kepler** y **Maxwell** pero que ha ido mejorando (por ejemplo añadiendo [page faulting](https://en.wikipedia.org/wiki/Page_fault) and [migration](https://www.kernel.org/doc/html/latest/vm/page_migration.html)) en las arquitecturas siguientes a la de *Kepler*: la arquitectura Pascal y Volta. Por esto en el *output* anterior de *nvprof* aparece una sección de *page fault*. 

* Obsérvese que en el programa anterior se comenta que se asume que $N$ el número de datos en el arreglo es menor al número de *threads* que es posible lanzar. Esto como veremos más adelante es importante considerar pues aunque en el *device* se pueden lanzar muchos bloques y muchos threads, se tienen límites en el número de éstos que es posible lanzar.


### ¿Tenemos que inicializar los datos en la CPU y copiarlos hacia la GPU?

En realidad no tenemos que realizarlo para el ejemplo de `suma_vectorial.cu` anterior. Por ejemplo:

In [26]:
%%file suma_vectorial_3.cu
#include<stdio.h>
#define N 10
__global__ void init(int *a, int *b){
    int thread_id_x = threadIdx.x;
    a[thread_id_x]=thread_id_x;
    b[thread_id_x]=thread_id_x*thread_id_x;
}

__global__ void suma_vect(int *a, int *b, int *c){
    int thread_id_x = threadIdx.x;
    if(thread_id_x<N) //aquí se asume que el valor de N 
                     //es menor al número máximo de threads que se pueden lanzar
                    //si fuese mayor, hay que hacer un ajuste
        c[thread_id_x] = a[thread_id_x]+b[thread_id_x];
}

int main(void){
    int *device_a, *device_b, *device_c;
    int i;
    dim3 dimGrid(1,1,1); //1 bloques en el grid
    dim3 dimBlock(N,1,1); //N threads por bloque 
    //alojando en device con Unified Memory
    cudaMallocManaged(&device_a, sizeof(int)*N);
    cudaMallocManaged(&device_b, sizeof(int)*N);
    cudaMallocManaged(&device_c, sizeof(int)*N);
    //llenando los arreglos:
    init<<<dimGrid,dimBlock>>>(device_a,device_b);
    cudaDeviceSynchronize();
    //llamando al kernel suma_vect
    suma_vect<<<dimGrid,dimBlock>>>(device_a,device_b,device_c); //1 bloque con N threads
    cudaDeviceSynchronize();
    for(i=0;i<N;i++)
        printf("%d+%d = %d\n",device_a[i],device_b[i],device_c[i]);
    cudaFree(device_a);
    cudaFree(device_b);
    cudaFree(device_c);
    return 0;
}

Writing suma_vectorial_3.cu


In [27]:
%%bash
nvcc --compiler-options -Wall suma_vectorial_3.cu -o suma_vectorial_3.out

In [28]:
%%bash
nvprof --normalized-time-unit s ./suma_vectorial_3.out

0+0 = 0
1+1 = 2
2+4 = 6
3+9 = 12
4+16 = 20
5+25 = 30
6+36 = 42
7+49 = 56
8+64 = 72
9+81 = 90


==436== NVPROF is profiling process 436, command: ./suma_vectorial_3.out
==436== Profiling application: ./suma_vectorial_3.out
==436== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
                        %         s                   s         s         s
 GPU activities:    51.28  1.92e-06         1  1.92e-06  1.92e-06  1.92e-06  suma_vect(int*, int*, int*)
                    48.72  1.82e-06         1  1.82e-06  1.82e-06  1.82e-06  init(int*, int*)
      API calls:    99.37  0.149377         3  0.049792  1.17e-05  0.149330  cudaMallocManaged
                     0.28  4.28e-04         2  2.14e-04  1.33e-05  4.15e-04  cudaLaunchKernel
                     0.19  2.93e-04        97  3.02e-06  3.01e-07  1.60e-04  cuDeviceGetAttribute
                     0.07  1.05e-04         3  3.50e-05  1.25e-05  6.47e-05  cudaFree
                     0.05  7.66e-05         1  7.66e-05  7.66e-05  7.66e-05  cuDeviceTotalMem
                     0.

## Arquitectura de una GPU y límites en número de threads y bloques que podemos lanzar en el kernel

## [CuPy](https://github.com/cupy/cupy)

## [Gputools](https://github.com/nullsatz/gputools) 

Ver [gputools: cran](https://rdrr.io/cran/gputools/)

## Referencias de interés

Para más sobre *Unified Memory* revisar:

* [Even easier introduction to cuda](https://devblogs.nvidia.com/even-easier-introduction-cuda/)

* [Unified memory cuda beginners](https://devblogs.nvidia.com/unified-memory-cuda-beginners/)

Es importante el manejo de errores por ejemplo en el alojamiento de memoria en la GPU. En este caso es útil revisar:

* [How to Query Device Properties and Handle Errors in CUDA C/C++](https://devblogs.nvidia.com/how-query-device-properties-and-handle-errors-cuda-cc/)

En las siguientes preguntas encontramos a personas desarrolladoras de CUDA que las resuelven y resultan muy útiles para continuar con el aprendizaje de CUDA C. Por ejemplo: 

* [Parallel reduction over one axis](https://stackoverflow.com/questions/51526082/cuda-parallel-reduction-over-one-axis)

Otros sistemas de software para el [Heterogeneous computing](https://en.wikipedia.org/wiki/Heterogeneous_computing) son:

* [OpenCl](https://en.wikipedia.org/wiki/OpenCL). Ver [NVIDIA OpenCL SDK Code Samples](https://developer.nvidia.com/opencl) para ejemplos con NVIDIA GPU's.

* [Rth-org/Rth](https://github.com/Rth-org/Rth) y más reciente [matloff/Rth](https://github.com/matloff/Rth). Ver también [rdrr.io matloff/Rth](https://rdrr.io/github/matloff/Rth/f/README.md).

Es posible escribir [kernels](https://github.com/ITAM-DS/analisis-numerico-computo-cientifico/blob/master/temas/II.computo_paralelo/2.3.CUDA_C.ipynb) con CuPy. Ver por ejemplo: [User-Defined Kernels](https://docs-cupy.chainer.org/en/stable/tutorial/kernel.html).

* [CuPy – NumPy-like API accelerated with CUDA](https://docs-cupy.chainer.org/en/stable/index.html)

* [CuPy : NumPy-like API accelerated with CUDA github](https://github.com/cupy/cupy)

Otro paquete para uso de Python+GPU para cómputo matricial es:

* [PyCUDA](https://github.com/inducer/pycuda/) y ver [PyCUDA en el repo de la clase](https://github.com/ITAM-DS/analisis-numerico-computo-cientifico/tree/master/Python/PyCUDA) para más información.

Un paquete para uso de pandas+GPU:

* [Rapids](https://github.com/rapidsai), [cudf](https://github.com/rapidsai/cudf)

Ver [optional-libraries](https://docs-cupy.chainer.org/en/stable/install.html#optional-libraries) para librerías que pueden ser utilizadas con CuPy.


```{admonition} Ejercicios
:class: tip

1.Resuelve los ejercicios y preguntas de la nota.
```


**Preguntas de comprehensión:**

1)¿Qué factores han determinado un mejor *performance* de una GPU vs una CPU? (contrasta los diseños de una CPU vs una GPU).

2)¿Dentro de qué modelo de arquitectura de máquinas se ubica a la GPU dentro de la taxonomía de Flynn? (tip: tal modelo se le puede comparar con el modelo **Single Program Multiple Data (SPMD)**)

3)¿Qué significan las siglas CUDA y detalla qué es CUDA?.

4)¿Qué es y en qué consiste CUDA C?
    
5)¿Qué es un *kernel*?

6)¿Qué pieza de CUDA se encarga de asignar los bloques de *cuda-threads* a las SM’s?

7)¿Qué características (recursos compartidos, dimensiones, forma de agendar la ejecución en *threads*) tienen los bloques que se asignan a una SM al lanzarse y ejecutarse un *kernel*?

8)¿Qué es un *warp*?

9)Menciona los tipos de memorias que existen en las GPU’s.

10)Supón que tienes una tarjeta GT200 cuyas características son:

* Máximo número de *threads* que soporta una SM en un mismo instante en el tiempo: 1024
* Máximo número de *threads* en un bloque: 512
* Máximo número de bloques por SM: 8
* Número de SM’s que tiene esta GPU: 30

Responde:

a)¿Cuál es la máxima cantidad de *threads* que puede soportar esta GPU en un mismo instante en el tiempo?

b)¿Cuál es la máxima cantidad de *warps* por SM que puede soportar esta GPU en un mismo instante en el tiempo?

c)¿Cuáles configuraciones de bloques y *threads* siguientes aprovechan la máxima cantidad de *warps* en una SM de esta GPU para un mismo instante en el tiempo?
    
1.Una configuración del tipo: bloques de 64 *threads* y 16 bloques.

2.Una configuración del tipo: bloques de 1024 *threads* y 1 bloque.

3.Una configuración del tipo: bloques de 256 *threads* y 4 bloques.

4.Una configuración del tipo: bloques de 512 *threads* y 8 bloques.

\*Debes considerar las restricciones/características de la GPU dadas para responder pues algunas configuraciones infringen las mismas. No estamos considerando *registers* o *shared memory*.


**Referencias:**

1. N. Matloff, Parallel Computing for Data Science. With Examples in R, C++ and CUDA, 2014.

2. D. B. Kirk, W. W. Hwu, Programming Massively Parallel Processors: A Hands-on Approach, Morgan Kaufmann, 2010.

3. NVIDIA,CUDA Programming Guide, NVIDIA Corporation, 2007.

4. B. W. Kernighan, D. M. Ritchie, The C Programming Language, Prentice Hall Software Series, 1988

5. [C/extensiones_a_C/CUDA/](https://github.com/palmoreck/programming-languages/tree/master/C/extensiones_a_C/CUDA)